In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import  Keys 
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.wait import WebDriverWait 
from selenium.webdriver.common.by import  By 
from selenium.webdriver.support import  expected_conditions as EC 

import pandas as pd
import numpy as np
import time
import datetime

In [9]:
#建立数据库
columns=['时间','发布日期','作者名','作者类型','粉丝数','投稿数','视频名','分区','子分区','视频类型','自制or转载','标签','视频长度','播放量','点赞数','投币数','收藏数','转发数','评论数','弹幕数','有无分集']
channel_youxi_wangluo=pd.DataFrame(columns=columns)
channel_youxi_wangluo

,时间,发布日期,作者名,作者类型,粉丝数,投稿数,视频名,分区,子分区,视频类型,...,标签,视频长度,播放量,点赞数,投币数,收藏数,转发数,评论数,弹幕数,有无分集


In [14]:
driver=webdriver.Chrome()
driver.get('https://passport.bilibili.com/login') 

In [15]:
#登录
driver.find_element_by_xpath('//*[@id="login-username"]').send_keys('15717506191')
driver.find_element_by_xpath('//*[@id="login-passwd"]').send_keys('xy000000')
driver.find_element_by_xpath('//*[@id="geetest-wrap"]/div/div[5]/a[1]').click()
#需要手动过验证码

In [16]:
driver.find_element_by_xpath('//*[@id="primaryChannelMenu"]/span[6]/div/a/span').click()  #选择频道

In [17]:
#获得分区名
channel=driver.find_element_by_xpath('//*[@id="primaryChannelMenu"]/span[6]/div/a/span').text
channel=channel.strip('0123456789+')
channel

'游戏'

In [18]:
driver.find_element_by_xpath('//*[@id="subnav"]/ul/li[5]/a').click()  #选择子频道
sub_channel=driver.find_element_by_xpath('//*[@id="subnav"]/ul/li[5]/a').text
sub_channel

'网络游戏'

In [19]:
driver.execute_script('window.scrollTo(0, document.body.scrollHeight)')        
driver.find_element_by_xpath('//*[@id="videolist_box"]/div[1]/div[1]/ul/a[2]/li').click() #让视频按热度排序
driver.find_element_by_xpath('//*[@id="videolist_box"]/div[1]/div[2]/ul[2]/li[1]').click() #更换视频放置界面

In [29]:
print(datetime.datetime.now())

for month in range(1,11): 
    driver.find_element_by_xpath('//*[@id="videolist_box"]/div[1]/div[3]/div[2]/div/span').click()
    period=driver.find_element_by_xpath('//*[@id="videolist_box"]/div[1]/div[3]/div[2]/div[2]/div[2]/div/div[2]/a[%d]'%month).text
    driver.find_element_by_xpath('//*[@id="videolist_box"]/div[1]/div[3]/div[2]/div[2]/div[2]/div/div[2]/a[%d]'%month).click()
    WebDriverWait(driver,60).until(EC.presence_of_element_located((By.XPATH,'//*[@id="videolist_box"]/div[2]/ul/li[1]/div[2]/div[3]/a'))) 
    for page in range(5):    #爬取每个频道1月到10月每个月播放量前100的视频

        video_loc=1
        while video_loc<21:

            ### 判断作者类型###
            
            
            driver.find_element_by_xpath('//*[@id="videolist_box"]/div[2]/ul/li[%d]/div[2]/div[3]/a'%video_loc).click()
            driver.switch_to_window(driver.window_handles[-1])
            
            
            author_web_show='No'
            try:
                i=0
                while author_web_show=='No':
                    driver.refresh()
                    i+=1
                    try:
                        WebDriverWait(driver,5).until(EC.presence_of_element_located((By.XPATH,'//*[@id="n-fs"]')))  #看作者名是否出现
                        WebDriverWait(driver,5).until(EC.presence_of_element_located((By.XPATH,'//*[@id="navigator"]/div/div[1]/div[1]/a[3]/span[3]')))  #看投稿数是否出现
                        author_web_show='Yes'
                    except:
                        author_web_show='No' 
                        if i>3:  
                            raise Exception  #刷新几次后仍爬不了说明用户已注销

                try:
                    driver.find_element_by_xpath('//*[@class="user-auth-subscript avatar-m organization-auth"]')
                    author_type='机构认证'

                except:
                    try:
                        driver.find_element_by_xpath('//*[@class="user-auth-subscript avatar-m personal-auth"]')
                        author_type='个人认证'
                    except:
                        author_type='无'

                fans_num=driver.find_element_by_xpath('//*[@id="n-fs"]').text  #获取作者粉丝数

                ### 获取作者投稿数 ###
                driver.find_element_by_xpath('//*[@class="n-num"]').click()
                WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,'//*[@id="page-video"]/div/div[1]/div/div/ul/li[1]/span')))  
                video_num=driver.find_element_by_xpath('//*[@id="page-video"]/div/div[1]/div/div/ul/li[1]/span').text

                driver.close()
                driver.switch_to_window(driver.window_handles[-1])

                view_num=driver.find_element_by_xpath('//*[@id="videolist_box"]/div[2]/ul/li[%d]/div[2]/div[2]/span[1]/span'%video_loc).text #获取播放量
                real_dan_mu_num=driver.find_element_by_xpath('//*[@id="videolist_box"]/div[2]/ul/li[%d]/div[2]/div[2]/span[2]/span'%video_loc).text #获取弹幕数
                author=driver.find_element_by_xpath('//*[@id="videolist_box"]/div[2]/ul/li[%d]/div[2]/div[3]/a'%video_loc).text #获取作者名
                release_date=driver.find_element_by_xpath('//*[@id="videolist_box"]/div[2]/ul/li[1]/div[2]/div[3]/span').text
                
                driver.find_element_by_xpath('//*[@id="videolist_box"]/div[2]/ul/li[%d]/div[2]/a'%video_loc).click() #进入视频界面  
                driver.switch_to_window(driver.window_handles[-1]) 

                try:
                    WebDriverWait(driver,5).until(EC.presence_of_element_located((By.XPATH,'//*[@id="viewbox_report"]/h1/span')))  #若出现视频被删除的情况则退出

                    video_web_show='No'
                    while video_web_show=='No':
                        driver.refresh()

                        try:
                            WebDriverWait(driver,60).until(EC.presence_of_element_located((By.XPATH,'//*[@class="bilibili-player-video-time-total"]'))) #以视频长度出现时间判定网页是否加载完
                            video_web_show='Yes'

                            try:      
                                driver.find_element_by_xpath('//*[@id="multi_page"]')   #测试是否有分集视频
                                multi='是'

                            except:
                                multi='否'

                            driver.execute_script('window.scrollTo(0, document.body.scrollHeight)')        
                            driver.execute_script('window.scrollTo(document.body.scrollHeight,0)') #再调回最上面是因为，视频长度的元素必须在屏幕内才可以被提取，否则为空

                            bv=driver.current_url[33:44] # 获取BV号（BV号在网址中）
                            print('开始BV号为%s视频属性的抓取'%bv)
                            video_name=driver.find_element_by_xpath('//*[@id="viewbox_report"]/h1/span').text #获取视频名

                            ### 获取视频类型 ###
                            try:
                                video_type=driver.find_element_by_xpath('//*[@class="activity"]').text
                                try:
                                    driver.find_element_by_xpath('//*[@id="app"]/div[2]/div[2]/div[1]/div[1]/span[1]')
                                    video_type=video_type+'和合作视频'
                                except:
                                    pass
                            except:
                                try:
                                    driver.find_element_by_xpath('//*[@id="app"]/div[2]/div[2]/div[1]/div[1]/span[1]')
                                    video_type='合作视频'
                                except:
                                    video_type='普通视频'


                            ### 获取点赞数 ###
                            if driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[1]').text=='点赞':
                                like_num=0
                            else:
                                like_num=driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[1]').text 

                            ### 获取投币数 ###
                            if driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[2]').text=='投币':
                                coin_num=0
                            else:
                                coin_num=driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[2]').text

                            ### 获取收藏数 ###             
                            if driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[3]').text=='收藏':
                                collect_num=0
                            else:
                                collect_num=driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[3]').text

                            ### 获取转发数 ###       
                            if driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[4]').text=='分享':
                                forward_num=0
                            else:
                                forward_num=driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[4]').text

                            ### 获取视频长度 ###

                            video_length=driver.find_element_by_xpath('//*[@class="bilibili-player-video-time-total"]').text  

                            ### 判断自制还是转载 ###
                            driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[2]').click()
                            try:
                                driver.find_element_by_xpath('//*[@id="app"]/div[2]/div[3]/div/div/div[2]/div[2]').text
                                self_made='自制'
                            except:
                                self_made='转载'
                            driver.find_element_by_xpath('//*[@id="app"]/div[2]/div[3]/div/div/i').click()

                            ### 获取标签 ###
                            k=1
                            labels=np.array
                            while True:
                                try:
                                    labels=np.append(labels,driver.find_element_by_xpath('//*[@id="v_tag"]/ul/li[%d]'%k).text)
                                    k+=1
                                except: 
                                    break
                            labels=labels[1:]

                            ### 获取评论数 ###
                            driver.execute_script('window.scrollTo(0, document.body.scrollHeight)')        
                            WebDriverWait(driver,20).until(EC.presence_of_element_located((By.XPATH,'//*[@id="comment"]/div/div[1]/span[2]'))) #保证"评论"二字能加载出来                                                   
                            WebDriverWait(driver,20).until(EC.presence_of_element_located((By.XPATH,'//*[@id="comment"]/div/div[2]/div/div[4]/div')))
                            if driver.find_element_by_xpath('//*[@id="comment"]/div/div[2]/div/div[4]/div').text=='没有更多信息':    #应对评论区被关闭的情况
                                comment_num='NaN'
                            else:
                                while driver.find_element_by_xpath('//*[@id="comment"]/div/div[1]/span[1]').text=='':
                                    try:
                                        WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,'//*[@id="comment"]/div/div[2]/div/div[4]/div[1]')))
                                        while driver.find_element_by_xpath('//*[@id="comment"]/div/div[1]/span[1]').text=='':
                                            time.sleep(0.1) #很奇怪，评论数可以定位的一瞬间并不能爬取，会返回空
                                        comment_num=driver.find_element_by_xpath('//*[@id="comment"]/div/div[1]/span[1]').text     
                                    except:
                                        driver.refresh()    #有的时候“评论”二字出现，评论数不出现，页面好像卡住了
                                        WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,'//*[@id="viewbox_report"]/h1/span')))
                                        driver.execute_script('window.scrollTo(0, document.body.scrollHeight)')
                                        WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,'//*[@id="comment"]/div/div[1]/span[1]')))

                                comment_num=driver.find_element_by_xpath('//*[@id="comment"]/div/div[1]/span[1]').text

                            channel_youxi_wangluo.loc[bv]=[period,release_date,author,author_type,fans_num,video_num,video_name,channel,sub_channel,video_type,self_made,labels,video_length,view_num,like_num,coin_num,collect_num,forward_num,comment_num,real_dan_mu_num,multi]

                            print('BV号为%s的视频属性抓取结束'%bv)

                            driver.close()
                            driver.switch_to_window(driver.window_handles[-1])

                        except:
                            video_web_show='No' 
                except:
                    driver.close()
                    driver.switch_to_window(driver.window_handles[-1])
            except:
                driver.close()
                driver.switch_to_window(driver.window_handles[-1])
            
            video_loc+=1


        driver.find_element_by_xpath('//*[@class="nav-btn iconfont icon-arrowdown3"]').click() #点击下一页
        WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,'//*[@id="videolist_box"]/div[2]/ul/li[1]')))  #点击下一页后等页面刷新
    driver.find_element_by_xpath('//*[@id="videolist_box"]/div[2]/div[2]/ul/li[2]').click()  #回到第1页
    WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,'//*[@id="videolist_box"]/div[2]/div[2]/ul/li[1]')))
    driver.find_element_by_xpath('//*[@id="videolist_box"]/div[2]/div[2]/ul/li[1]').click()  #发现很多月份漏爬了约20个，而且恰好是第一页，所以再点一次第一页
        
        
print(datetime.datetime.now()) 
    


2020-12-15 22:55:52.983332


<ipython-input-29-a17ccd8f12f2>:17: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(driver.window_handles[-1])
<ipython-input-29-a17ccd8f12f2>:54: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(driver.window_handles[-1])
<ipython-input-29-a17ccd8f12f2>:62: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(driver.window_handles[-1])


开始BV号为1MJ411H77i视频属性的抓取
BV号为1MJ411H77i的视频属性抓取结束


<ipython-input-29-a17ccd8f12f2>:179: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(driver.window_handles[-1])


开始BV号为1oJ411V7s1视频属性的抓取
BV号为1oJ411V7s1的视频属性抓取结束
开始BV号为1D7411B7kS视频属性的抓取
BV号为1D7411B7kS的视频属性抓取结束
开始BV号为1v7411v7Fg视频属性的抓取
BV号为1v7411v7Fg的视频属性抓取结束
开始BV号为1pJ41177uk视频属性的抓取
BV号为1pJ41177uk的视频属性抓取结束
开始BV号为1U7411k7DW视频属性的抓取
BV号为1U7411k7DW的视频属性抓取结束
开始BV号为1w741167D1视频属性的抓取
BV号为1w741167D1的视频属性抓取结束
开始BV号为1hJ411b772视频属性的抓取
BV号为1hJ411b772的视频属性抓取结束
开始BV号为1u7411k77v视频属性的抓取
BV号为1u7411k77v的视频属性抓取结束
开始BV号为14J411J7pm视频属性的抓取
BV号为14J411J7pm的视频属性抓取结束
开始BV号为1R7411q7ad视频属性的抓取
BV号为1R7411q7ad的视频属性抓取结束
开始BV号为1GJ411V7bh视频属性的抓取
BV号为1GJ411V7bh的视频属性抓取结束
开始BV号为197411e7pC视频属性的抓取
BV号为197411e7pC的视频属性抓取结束
开始BV号为1yJ411j7DJ视频属性的抓取
BV号为1yJ411j7DJ的视频属性抓取结束
开始BV号为1K7411e7UX视频属性的抓取
BV号为1K7411e7UX的视频属性抓取结束
开始BV号为1GJ411n7BE视频属性的抓取
BV号为1GJ411n7BE的视频属性抓取结束
开始BV号为1BJ411V7wV视频属性的抓取
BV号为1BJ411V7wV的视频属性抓取结束
开始BV号为1p7411i7MA视频属性的抓取
BV号为1p7411i7MA的视频属性抓取结束
开始BV号为1e7411Y71d视频属性的抓取
BV号为1e7411Y71d的视频属性抓取结束
开始BV号为1kJ411n7nX视频属性的抓取
BV号为1kJ411n7nX的视频属性抓取结束
开始BV号为1Q7411Y7R6视频属性的抓取
BV号为1Q7411Y7R6的视频属性抓取结束
开始BV号为1kJ411L7TE视频属性的抓取
BV号为1kJ411L7TE的视

...

开始BV号为1zK411n7wb视频属性的抓取
BV号为1zK411n7wb的视频属性抓取结束
开始BV号为1fA411q7rp视频属性的抓取
BV号为1fA411q7rp的视频属性抓取结束
开始BV号为1hQ4y1P7cM视频属性的抓取
BV号为1hQ4y1P7cM的视频属性抓取结束
开始BV号为1Ma4y1Y7Qf视频属性的抓取
BV号为1Ma4y1Y7Qf的视频属性抓取结束
开始BV号为17z411i7Pf视频属性的抓取
BV号为17z411i7Pf的视频属性抓取结束
开始BV号为14p4y1D7r7视频属性的抓取
BV号为14p4y1D7r7的视频属性抓取结束
开始BV号为1kV411r7BR视频属性的抓取
BV号为1kV411r7BR的视频属性抓取结束
开始BV号为1CA411v794视频属性的抓取
BV号为1CA411v794的视频属性抓取结束
开始BV号为1qz4y197rv视频属性的抓取
BV号为1qz4y197rv的视频属性抓取结束
开始BV号为1Lv411B7cg视频属性的抓取
BV号为1Lv411B7cg的视频属性抓取结束
开始BV号为1ia4y1v7CV视频属性的抓取
BV号为1ia4y1v7CV的视频属性抓取结束
开始BV号为15i4y1G7mw视频属性的抓取
BV号为15i4y1G7mw的视频属性抓取结束
开始BV号为1rp4y1D7Jz视频属性的抓取
BV号为1rp4y1D7Jz的视频属性抓取结束
开始BV号为1AK411W7fr视频属性的抓取
BV号为1AK411W7fr的视频属性抓取结束
开始BV号为1pi4y1G7Kj视频属性的抓取
BV号为1pi4y1G7Kj的视频属性抓取结束
开始BV号为1HK411p7D5视频属性的抓取
BV号为1HK411p7D5的视频属性抓取结束
开始BV号为1eK4y1t7qt视频属性的抓取
BV号为1eK4y1t7qt的视频属性抓取结束
开始BV号为1cZ4y1H7Xs视频属性的抓取
BV号为1cZ4y1H7Xs的视频属性抓取结束
开始BV号为1Wa4y1v72N视频属性的抓取
BV号为1Wa4y1v72N的视频属性抓取结束
开始BV号为16K411p7nc视频属性的抓取
BV号为16K411p7nc的视频属性抓取结束
开始BV号为1QA411v7Vm视频属性的抓取
BV号为1QA411v7Vm的视

开始BV号为1zv411i7UR视频属性的抓取
BV号为1zv411i7UR的视频属性抓取结束
开始BV号为1Ak4y1B77W视频属性的抓取
BV号为1Ak4y1B77W的视频属性抓取结束
开始BV号为1nV41167hh视频属性的抓取
BV号为1nV41167hh的视频属性抓取结束
开始BV号为1K5411Y7rQ视频属性的抓取
BV号为1K5411Y7rQ的视频属性抓取结束
开始BV号为1w54y1z7wR视频属性的抓取
BV号为1w54y1z7wR的视频属性抓取结束
开始BV号为1cz4y1X7Cc视频属性的抓取
BV号为1cz4y1X7Cc的视频属性抓取结束
开始BV号为1DT4y1J7NV视频属性的抓取
BV号为1DT4y1J7NV的视频属性抓取结束
开始BV号为1CK4y147tu视频属性的抓取
BV号为1CK4y147tu的视频属性抓取结束
开始BV号为1vC4y1h7sM视频属性的抓取
BV号为1vC4y1h7sM的视频属性抓取结束
开始BV号为1st4y1D7hx视频属性的抓取
BV号为1st4y1D7hx的视频属性抓取结束
开始BV号为1ap4y1S7pL视频属性的抓取
BV号为1ap4y1S7pL的视频属性抓取结束
开始BV号为1eK411J7Fw视频属性的抓取
BV号为1eK411J7Fw的视频属性抓取结束
开始BV号为1jC4y187Gg视频属性的抓取
BV号为1jC4y187Gg的视频属性抓取结束
开始BV号为1CD4y1S7kC视频属性的抓取
BV号为1CD4y1S7kC的视频属性抓取结束
开始BV号为15k4y1m7Qp视频属性的抓取
BV号为15k4y1m7Qp的视频属性抓取结束
开始BV号为12A411i7uH视频属性的抓取
BV号为12A411i7uH的视频属性抓取结束
开始BV号为1Di4y1G7Kp视频属性的抓取
BV号为1Di4y1G7Kp的视频属性抓取结束
开始BV号为1CK4y1s7mK视频属性的抓取
BV号为1CK4y1s7mK的视频属性抓取结束
开始BV号为1Hz4y1D7UT视频属性的抓取
BV号为1Hz4y1D7UT的视频属性抓取结束
开始BV号为1eZ4y1u7Nf视频属性的抓取
BV号为1eZ4y1u7Nf的视频属性抓取结束
开始BV号为1GV411r7gU视频属性的抓取
BV号为1GV411r7gU的视

<ipython-input-29-a17ccd8f12f2>:188: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(driver.window_handles[-1])


开始BV号为13t4y1Q7HC视频属性的抓取
BV号为13t4y1Q7HC的视频属性抓取结束
开始BV号为17C4y1t7se视频属性的抓取
BV号为17C4y1t7se的视频属性抓取结束
开始BV号为1oK4y1v7Ky视频属性的抓取
BV号为1oK4y1v7Ky的视频属性抓取结束
开始BV号为1QA411a7vE视频属性的抓取
BV号为1QA411a7vE的视频属性抓取结束
开始BV号为1da4y177sU视频属性的抓取
BV号为1da4y177sU的视频属性抓取结束
开始BV号为1ot4y1U7oV视频属性的抓取
BV号为1ot4y1U7oV的视频属性抓取结束
开始BV号为13D4y1o7y5视频属性的抓取
BV号为13D4y1o7y5的视频属性抓取结束
开始BV号为1oA411E7rU视频属性的抓取
BV号为1oA411E7rU的视频属性抓取结束
开始BV号为1ez4y1Z7rr视频属性的抓取
BV号为1ez4y1Z7rr的视频属性抓取结束
开始BV号为1Dt4y1i76t视频属性的抓取
BV号为1Dt4y1i76t的视频属性抓取结束
开始BV号为1Xt4y1i7gk视频属性的抓取
BV号为1Xt4y1i7gk的视频属性抓取结束
开始BV号为1rv411C77n视频属性的抓取
BV号为1rv411C77n的视频属性抓取结束
开始BV号为14f4y1X7ka视频属性的抓取
BV号为14f4y1X7ka的视频属性抓取结束
开始BV号为13a4y1E7AS视频属性的抓取
BV号为13a4y1E7AS的视频属性抓取结束
开始BV号为1xk4y117tg视频属性的抓取
BV号为1xk4y117tg的视频属性抓取结束
开始BV号为1JD4y1o74G视频属性的抓取
BV号为1JD4y1o74G的视频属性抓取结束
开始BV号为1nK4y1a7vr视频属性的抓取
BV号为1nK4y1a7vr的视频属性抓取结束
开始BV号为1cT4y1A7fP视频属性的抓取
BV号为1cT4y1A7fP的视频属性抓取结束
开始BV号为1gt4y1q7YV视频属性的抓取
BV号为1gt4y1q7YV的视频属性抓取结束
开始BV号为1mf4y1D7dz视频属性的抓取
BV号为1mf4y1D7dz的视频属性抓取结束
开始BV号为1ka4y1j7Rk视频属性的抓取
BV号为1ka4y1j7Rk的视

BV号为1Vh41197co的视频属性抓取结束
开始BV号为1Hp4y1Y7TG视频属性的抓取
BV号为1Hp4y1Y7TG的视频属性抓取结束
开始BV号为19K4y177JQ视频属性的抓取
BV号为19K4y177JQ的视频属性抓取结束
开始BV号为1v54y1R7GV视频属性的抓取
BV号为1v54y1R7GV的视频属性抓取结束
开始BV号为1ni4y1772e视频属性的抓取
BV号为1ni4y1772e的视频属性抓取结束
开始BV号为1Qt4y1v7wv视频属性的抓取
BV号为1Qt4y1v7wv的视频属性抓取结束
开始BV号为1op4y1a7EQ视频属性的抓取
BV号为1op4y1a7EQ的视频属性抓取结束
开始BV号为1w54y1C7cj视频属性的抓取
BV号为1w54y1C7cj的视频属性抓取结束
开始BV号为1Ci4y1j7Qs视频属性的抓取
BV号为1Ci4y1j7Qs的视频属性抓取结束
开始BV号为1Fa411P7Bg视频属性的抓取
BV号为1Fa411P7Bg的视频属性抓取结束
开始BV号为18t4y1i7pm视频属性的抓取
BV号为18t4y1i7pm的视频属性抓取结束
开始BV号为1af4y1i7gM视频属性的抓取
BV号为1af4y1i7gM的视频属性抓取结束
开始BV号为13a4y1L7bv视频属性的抓取
BV号为13a4y1L7bv的视频属性抓取结束
开始BV号为195411j7fx视频属性的抓取
BV号为195411j7fx的视频属性抓取结束
开始BV号为1PD4y1d7d6视频属性的抓取
BV号为1PD4y1d7d6的视频属性抓取结束
开始BV号为1Nz4y1o7jg视频属性的抓取
BV号为1Nz4y1o7jg的视频属性抓取结束
开始BV号为1AT4y1w7fz视频属性的抓取
BV号为1AT4y1w7fz的视频属性抓取结束
开始BV号为1SV41117jG视频属性的抓取
BV号为1SV41117jG的视频属性抓取结束
开始BV号为1WV411y7b4视频属性的抓取
BV号为1WV411y7b4的视频属性抓取结束
开始BV号为1JZ4y1573D视频属性的抓取
BV号为1JZ4y1573D的视频属性抓取结束
开始BV号为19p4y1a7TD视频属性的抓取
BV号为19p4y1a7TD的视频属性抓取结束
开始BV号为1Ra411w7PX

In [28]:
driver.switch_to_window(driver.window_handles[-1])

<ipython-input-28-dd0a1399d54d>:1: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(driver.window_handles[-1])


In [30]:
channel_youxi_wangluo.groupby('时间').agg('count')

,发布日期,作者名,作者类型,粉丝数,投稿数,视频名,分区,子分区,视频类型,自制or转载,标签,视频长度,播放量,点赞数,投币数,收藏数,转发数,评论数,弹幕数,有无分集
时间,,,,,,,,,,,,,,,,,,,,
10月,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100
1月,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100
2月,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100
3月,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100
4月,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100
5月,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100
6月,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100
7月,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100
8月,96,96,96,96,96,96,96,96,96,96,96,96,96,96,96,96,96,96,96,96


In [32]:
channel_youxi_wangluo.to_json('channel_youxi_wangluo.json')

In [17]:
pd.set_option('display.max_rows', None)

In [18]:
channel_youxi_danji[channel_youxi_danji['时间']=='4月']

,时间,作者名,作者类型,粉丝数,投稿数,视频名,分区,子分区,视频类型,自制or转载,标签,视频长度,播放量,点赞数,投币数,收藏数,转发数,评论数,弹幕数,有无分集
1h54y1d7Qq,4月,鹤吱菌,个人认证,191.3万,81,【师生对线】这网课上得就尼玛离谱！（第五集）,游戏,单机游戏,活动作品,自制,"[配音, 守望先锋, 独立游戏, 搞笑, 单机游戏, 游戏, 单机游戏, 电子竞技, 配音大...",13:00,953.7万,114.2万,115.2万,42.5万,8.9万,25302,18.8万,否
1L64y1M7Dp,4月,老番茄,个人认证,1360.1万,348,【老番茄】史上最骚骑士,游戏,单机游戏,活动作品,自制,"[骑马与砍杀, 搞笑, 游戏, 单机游戏, 老番茄, 骑马与砍杀2, 游戏练级挑战, 骑砍,...",15:45,902.3万,84.5万,61.7万,24.5万,2.4万,21336,9.7万,否
18K41157Tu,4月,老番茄,个人认证,1360.1万,348,宫斗大师老番茄,游戏,单机游戏,普通视频,自制,"[游戏, 单机游戏, 宫妃, 易次元]",13:55,739.6万,45.1万,9.3万,5.5万,1.3万,14189,10.6万,否
1PT4y1G7ME,4月,老番茄,个人认证,1360.1万,348,常胜将军老番茄,游戏,单机游戏,普通视频,自制,"[游戏, 单机游戏, 老番茄, shield wall]",14:54,726.8万,48.7万,12.9万,5.8万,7401,15180,7.0万,否
1S54y197uX,4月,徐大虾咯,个人认证,570.1万,224,家 里 有 矿 ！,游戏,单机游戏,普通视频,自制,"[我的世界, 单机游戏, 游戏, 单机游戏, 徐大虾咯, 徐大虾, MINECRAFT, M...",12:18,714.5万,35.0万,8.8万,4.3万,4860,7738,6.7万,否
1ti4y1t7YM,4月,老番茄,个人认证,1360.1万,348,解说挑战！什么游戏都能解说？,游戏,单机游戏,普通视频,自制,"[游戏, 单机游戏, 老番茄]",08:19,711.2万,61.4万,16.7万,8.1万,9970,14368,10.6万,否
1Ai4y1b7ET,4月,敖厂长,个人认证,735.1万,348,花了一万多元终于见到最早开放世界游戏之一,游戏,单机游戏,普通视频,自制,"[游戏, 单机游戏, 游戏资讯, 敖厂长, 单机, 游戏杂谈, 游戏视频, 杂谈]",14:26,670.4万,112.6万,156.8万,34.9万,2.4万,15618,8.6万,否
1Up4y1C7PB,4月,丢鹏D1uuu,无,8.4万,20,年级第一：别TM挡着我上网课！！！,游戏,单机游戏,活动作品,自制,"[独立游戏, 搞笑, 单机游戏, 游戏, 单机游戏, bilibili新星计划, 新人, 恶...",05:38,654.0万,17.3万,1.5万,2.0万,1321,1952,4526,否
1wi4y187bu,4月,中国BOY超级大猩猩,个人认证,663.5万,1380,全员铁憨憨!,游戏,单机游戏,活动作品,自制,"[STEAM, 单机游戏, 游戏, 单机游戏, 花少北, 老番茄, 中国BOY, LexBu...",19:57,653.2万,42.0万,23.9万,14.2万,2.5万,13493,9.7万,否
13t4y1U7kQ,4月,徐大虾咯,个人认证,570.1万,224,德华的世界#2,游戏,单机游戏,普通视频,自制,"[我的世界, 单机游戏, 游戏, 单机游戏, 徐大虾咯, 徐大虾, MINECRAFT, M...",12:07,619.7万,35.1万,9.5万,4.4万,7033,7937,6.3万,否


In [31]:
channel_youxi_wangluo

,时间,发布日期,作者名,作者类型,粉丝数,投稿数,视频名,分区,子分区,视频类型,...,标签,视频长度,播放量,点赞数,投币数,收藏数,转发数,评论数,弹幕数,有无分集
1MJ411H77i,1月,2020-01-14 15:10:24,原神,机构认证,428.3万,53,《原神》新地区预告-「浮世浮生千岩间」,游戏,网络游戏,普通视频,...,"[原神, 游戏, 网络游戏, 手机游戏, miHoYo, 香菱, 派蒙, 米哈游, PC, 璃月]",01:12,526.3万,5.1万,1.9万,1.1万,1.8万,11442,5698,是
1oJ411V7s1,1月,2020-01-14 15:10:24,丶小禾,无,1.1万,222,当火线妹关闭直播美颜功能以后，才知道她做主播有多幸苦！,游戏,网络游戏,普通视频,...,"[绝地求生, 吃鸡, 游戏, 网络游戏, 网络游戏, 游戏, 火线妹]",01:12,384.1万,3.7万,1164,6585,538,2051,2506,否
1D7411B7kS,1月,2020-01-14 15:10:24,小潮院长,个人认证,481.9万,266,小姐姐可爱哭了！老板当场就爱了…【小潮国基网骗】,游戏,网络游戏,普通视频,...,"[绝地求生, 鬼畜, 吃鸡, 搞笑, 游戏, 网络游戏, 小潮, 国际网骗, 萌妹, 高能,...",05:02,379.0万,23.0万,10.0万,4.7万,6213,2553,2.3万,否
1v7411v7Fg,1月,2020-01-14 15:10:24,槐安遗梦,个人认证,113.2万,92,百万粉up主就能绿我，抢我女朋友吗？,游戏,网络游戏,活动作品,...,"[绝地求生, 搞笑, 游戏, 网络游戏, 网骗, 全能打卡挑战, 网络游戏, 日常, 生活,...",08:31,364.6万,6.4万,1.9万,7029,1642,25193,4.3万,否
1pJ41177uk,1月,2020-01-14 15:10:24,槐安遗梦,个人认证,113.2万,92,悲惨老板花10万元点陪玩却被277名陪玩小姐姐喊渣男,游戏,网络游戏,普通视频,...,"[绝地求生, 搞笑, 游戏, 网络游戏, 绝地大逃杀, 网络游戏, 生活, 小姐姐]",07:17,235.6万,20.5万,15.8万,4.4万,3713,2699,8055,否
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1zZ4y157uu,10月,2020-10-28 19:09:20,猪酱的日常,个人认证,34.2万,1758,PDD找工具人刷原神最难副本：充了12万依旧刮痧？！,游戏,网络游戏,活动作品,...,"[PDD, 原神, 游戏, 网络游戏, 原神-游戏区征稿, 网络游戏, 游戏视频, 狼的末日...",26:05,41.6万,7378,138,388,739,1485,1132,否
1ak4y1C71b,10月,2020-10-28 19:09:20,PokeJeff,无,180,13,【原神】AOE之王 砂糖+菲谢尔,游戏,网络游戏,活动作品,...,"[原神, 游戏, 网络游戏, 原神-游戏区征稿, 网络游戏, 游戏视频, 元素, 单刷, P...",01:02,41.3万,4274,232,1059,2712,685,176,否
1yk4y1C7ic,10月,2020-10-28 19:09:20,再世梦,无,1913,220,【原神】每天损失一个亿，第一次刷虎牢山让我痛彻心扉！,游戏,网络游戏,活动作品,...,"[单机游戏, 游戏, 网络游戏, 搞笑剪辑, 攻略, 新人, 十月打卡挑战W2, 肝帝, 宝...",05:28,41.2万,6415,177,945,394,1357,439,否
1M54y1r7J5,10月,2020-10-28 19:09:20,PDD在B站,无,128.4万,1156,PDD：哎兄弟，你原神玩的这么专业要不要来我这上班啊？,游戏,网络游戏,普通视频,...,"[PDD, 原神, 英雄联盟, 游戏, 网络游戏, 电子竞技, 手机游戏, 手游, LOL,...",05:39,41.2万,9285,370,653,1179,873,395,否
